# Introduction


In this notebook we demostrate how to finetune **Llama3** (8B) model from Meta using **QLoRA** and **SFTTrainer** from **tlr**.

## What is Llama3?

Llama3 is the latest release of open-source LLMs from Meta, with features pretrained and instruction-fine-tuned language models with 8B and 70B parameters.

## What is LoRA?

LoRA stands for Low-Rank Adaptation. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices. The number of trainable parameters during fine-tunning will decrease therefore considerably. According to LoRA paper, this number decreases 10,000 times, and the computational resources size decreases 3 times.


## What is QLoRA?

QLoRA builds on LoRA by incorporating quantization techniques to further reduce memory usage while maintaining, or even enhancing, model performance. With QLoRA it is possible to finetune a 70B parameter model that requires 36 GPUs with only 2!


## What is PEFT?

Parameter-efficient Fine-tuning (PEFT) is a technique used in Natural Language Processing (NLP) to improve the performance of pre-trained language models on specific downstream tasks. It involves reusing the pre-trained model’s parameters and fine-tuning them on a smaller dataset, which saves computational resources and time compared to training the entire model from scratch. PEFT achieves this efficiency by freezing some of the layers of the pre-trained model and only fine-tuning the last few layers that are specific to the downstream task.

## What is SFTTrainer?

SFT in SFTTrainer stands for supervised fine-tuning. The **trl** (Transformer Reinforcement Learning) library from HuggingFace provides a simple API to fine-tune models using SFTTrainer.

## What is UltraChat200k?  

UltraChat-200k is an invaluable resource for natural language understanding, generation and dialog system research. With 1.4 million dialogues spanning a variety of topics, this parquet-formatted dataset offers researchers four distinct formats to aid in their studies: test_sft, train_sft, train_gen and test_gen. More details [here](https://www.kaggle.com/datasets/thedevastator/ultrachat-200k-nlp-dataset).

## Inspiration

For this notebook, I took inspiration from several sources:
* [Efficiently fine-tune Llama 3 with PyTorch FSDP and Q-Lora](https://www.philschmid.de/fsdp-qlora-llama3)  
* [Fine-tuning LLMs using LoRA](https://medium.com/@rajatsharma_33357/fine-tuning-llama-using-lora-fb3f48a557d5)  
* [Fine-tuning Llama-3–8B-Instruct QLORA using low cost resources](https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04)  
* [Llama2 Fine-Tuning with Low-Rank Adaptations (LoRA) on Gaudi 2 Processors](https://eduand-alvarez.medium.com/llama2-fine-tuning-with-low-rank-adaptations-lora-on-gaudi-2-processors-52cf1ee6ce11)  

# Install and import libraries

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb_api")
import wandb
! wandb login $wandb_key

In [ ]:
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer,setup_chat_format

# Initialize model


The model used is:

* **Model**: Llama3  
* **Framework**: Transformers   
* **Size**: 8B   
* **Type**: 8b-chat-hf (hf stands for HuggingFace). 
* **Version**: V1  

In [ ]:
model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

We are using quantization (with BitsAndBytes).

In [ ]:
compute_dtype = torch.bfloat16
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True)

We define the model configuration, the model (using AutoModelForCausalLM) and the tokenizer (using AutoTokenizer).

In [ ]:
time_start = time()

model_config = AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

In [ ]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

# Prepare the dataset   


We will use 10K rows from the `ultrachat_200k` database.

In [ ]:
dataset_name = "HuggingFaceH4/ultrachat_200k"
dataset = load_dataset(dataset_name, split="train_sft")
dataset = dataset.shuffle(seed=42).select(range(10000))

def format_chat_template(row):
    chat = tokenizer.apply_chat_template(row["messages"], tokenize=False)
    return {"text":chat}

processed_dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)

dataset = processed_dataset.train_test_split(test_size=0.01)

# Training

## Training configuration   

We set the rank for LoRA to 4, to reduce the number of trainable parameters.

In [ ]:
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=4,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results_llama3_sft/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=20,
        logging_steps=1,
        learning_rate=8e-6,
        eval_steps=1,
        max_steps=20,
        num_train_epochs=20,
        warmup_steps=3,
        lr_scheduler_type="linear",
)

## Training process

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

# Conclusions

By reducing the batch_size, max_seq_length and the LoRA rank, we managed to run the Llama3 fine-tunning with QLoRA in the Kaggle environment.

**Note**: we set `os.environ["WANDB_DISABLED"]` to `True` and `save_steps` to `20`. We intend to save the last checkpoint as a **Kaggle Model**.
